In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from tqdm import tqdm
from deep_translator import GoogleTranslator
from camel_tools.utils.normalize import normalize_alef_maksura_ar, normalize_teh_marbuta_ar
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score


c:\Users\noura\OneDrive\Desktop\university\arabic sentiment analysis project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df=pd.read_csv(r'C:\Users\noura\OneDrive\Desktop\university\arabic sentiment analysis project\data\elmenus_reviews.csv')
df.head()

,source,text,rating,reviewer,time_stamp
0,Elmenus,"the delivery was very very late, the estimated...",2,Judy Asal,8 days ago
1,Elmenus,Lots of sauce and not much beef. The photo sho...,2,Patrick bradley,15 days ago
2,Elmenus,No sushi menu!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! ...,1,Oleksandr Kumeiko,a month ago
3,Elmenus,الاكل بارد جدا واتاخر ساعه,1,Abdallah M. Ahmed,a month ago
4,Elmenus,الشانتونج كله لازق في بعضه,3,Amir Rafik,a month ago


In [3]:
print("Total reviews before removing duplicates:", len(df))
print("Duplicate reviews:", df.duplicated(subset=['text']).sum())


Total reviews before removing duplicates: 455
Duplicate reviews: 1


In [4]:
df=df.drop_duplicates()
print("Duplicate reviews:", df.duplicated(subset=['text']).sum())

Duplicate reviews: 0


In [5]:
print(f"Reviews for sentiment analysis: {len(df)}  ")

Reviews for sentiment analysis: 454  


In [6]:
# import re
# def contains_english(text):
#     return bool(re.search('[a-zA-Z]', str(text)))

# tqdm.pandas()

# def translate_to_arabic(text):
#     text = str(text)
#     if not text.strip():  
#         return text
#     if contains_english(text):
#         return GoogleTranslator(source='auto', target='ar').translate(text)
#     return text

# df['text_translated'] = df['text'].progress_apply(translate_to_arabic)
# df.to_csv("elmenus_reviews_translated.csv", index=False, encoding='utf-8')
# print("Translation complete. Saved as elmenus_reviews_translated.csv")

In [2]:
df=pd.read_csv(r"C:\Users\noura\OneDrive\Desktop\university\arabic sentiment analysis project\notebooks\elmenus_reviews_translated.csv")


In [3]:
df.head()

,source,text,rating,reviewer,time_stamp,text_translated
0,Elmenus,"the delivery was very very late, the estimated...",2,Judy Asal,8 days ago,كان التسليم متأخراً للغاية ، وكان الوقت المقدر...
1,Elmenus,Lots of sauce and not much beef. The photo sho...,2,Patrick bradley,15 days ago,الكثير من الصلصة وليس الكثير من اللحم البقري. ...
2,Elmenus,No sushi menu!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! ...,1,Oleksandr Kumeiko,a month ago,لا قائمة السوشي !!!!!! لا قائمة السوشي !!!!
3,Elmenus,الاكل بارد جدا واتاخر ساعه,1,Abdallah M. Ahmed,a month ago,الاكل بارد جدا واتاخر ساعه
4,Elmenus,الشانتونج كله لازق في بعضه,3,Amir Rafik,a month ago,الشانتونج كله لازق في بعضه


In [4]:
def rating_to_sentiment(rating):
    if rating <= 2:
        return 'negative'
    elif rating == 3:
        return 'neutral'
    else:
        return 'positive'
    
df['sentiment'] = df['rating'].apply(rating_to_sentiment)
df.to_csv("elmenus_reviews_with_sentiment.csv", index=False, encoding='utf-8')

In [5]:
df= pd.read_csv(r"C:\Users\noura\OneDrive\Desktop\university\arabic sentiment analysis project\notebooks\elmenus_reviews_with_sentiment.csv")


In [6]:
df.head()

,source,text,rating,reviewer,time_stamp,text_translated,sentiment
0,Elmenus,"the delivery was very very late, the estimated...",2,Judy Asal,8 days ago,كان التسليم متأخراً للغاية ، وكان الوقت المقدر...,negative
1,Elmenus,Lots of sauce and not much beef. The photo sho...,2,Patrick bradley,15 days ago,الكثير من الصلصة وليس الكثير من اللحم البقري. ...,negative
2,Elmenus,No sushi menu!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! ...,1,Oleksandr Kumeiko,a month ago,لا قائمة السوشي !!!!!! لا قائمة السوشي !!!!,negative
3,Elmenus,الاكل بارد جدا واتاخر ساعه,1,Abdallah M. Ahmed,a month ago,الاكل بارد جدا واتاخر ساعه,negative
4,Elmenus,الشانتونج كله لازق في بعضه,3,Amir Rafik,a month ago,الشانتونج كله لازق في بعضه,neutral


In [7]:
df['sentiment'].value_counts()

sentiment
negative    255
positive    121
neutral      78
Name: count, dtype: int64

In [8]:
def preprocess_arabic(text):
    text=str(text)
    text=normalize_alef_maksura_ar(text)
    text=normalize_teh_marbuta_ar(text)
    text = re.sub(r'[^\w\s]', '', text)  
    return text

df['text_translated'] = df['text_translated'].apply(preprocess_arabic)
df=df[['text_translated', 'sentiment']].dropna()

In [9]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


In [10]:
from datasets import Dataset
train_dataset=Dataset.from_pandas(train_df)
test_dataset=Dataset.from_pandas(test_df)

In [11]:
model_name="asafaya/bert-mini-arabic"
tokenizer= AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at asafaya/bert-mini-arabic and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
def tokenize_function(examples):
    return tokenizer(examples['text_translated'], 
                    padding='max_length', truncation=True, max_length=128)

In [13]:
train_dataset=train_dataset.map(tokenize_function, batched=True)
test_dataset=test_dataset.map(tokenize_function, batched=True)

label_map={'negative': 0,
           'neutral': 1,
           'positive': 2}
train_dataset = train_dataset.map(lambda x: {'label': label_map[x['sentiment']]})
test_dataset = test_dataset.map(lambda x: {'label': label_map[x['sentiment']]})


Map: 100%|██████████| 91/91 [00:00<00:00, 8018.19 examples/s]


In [14]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

classes = np.array(['negative', 'neutral', 'positive'])
y_train = train_df['sentiment'].values
class_weights = compute_class_weight('balanced', classes=classes, y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float)

In [15]:
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss


In [16]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,  # Increased for better convergence
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    learning_rate=3e-5,
    weight_decay=0.01,
    fp16=False,
    no_cuda=True,
)

c:\Users\noura\OneDrive\Desktop\university\arabic sentiment analysis project\.venv\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\noura\OneDrive\Desktop\university\arabic sentiment analysis project\.venv\Lib\site-packages\transformers\training_args.py:1540: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [17]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {'accuracy': acc, 'f1': f1}

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [19]:
trainer.train()
eval_results = trainer.evaluate()
print(f"Accuracy: {eval_results['eval_accuracy']:.2f}, F1-Score: {eval_results['eval_f1']:.2f}")

                                                
 20%|██        | 45/225 [00:20<01:16,  2.34it/s]

{'eval_loss': 0.8877268433570862, 'eval_accuracy': 0.6043956043956044, 'eval_f1': 0.45536655125696224, 'eval_runtime': 0.7955, 'eval_samples_per_second': 114.398, 'eval_steps_per_second': 28.914, 'epoch': 0.99}


                                                
 40%|████      | 91/225 [00:38<00:44,  3.01it/s]

{'eval_loss': 0.8472296595573425, 'eval_accuracy': 0.6813186813186813, 'eval_f1': 0.5984909874298364, 'eval_runtime': 0.7482, 'eval_samples_per_second': 121.626, 'eval_steps_per_second': 30.741, 'epoch': 2.0}


                                                 
 60%|██████    | 136/225 [00:55<00:33,  2.67it/s]

...

{'eval_loss': 0.8153166770935059, 'eval_accuracy': 0.7362637362637363, 'eval_f1': 0.6827402248913876, 'eval_runtime': 0.8265, 'eval_samples_per_second': 110.101, 'eval_steps_per_second': 27.828, 'epoch': 4.95}


100%|██████████| 225/225 [01:29<00:00,  2.51it/s]


{'train_runtime': 89.6264, 'train_samples_per_second': 20.251, 'train_steps_per_second': 2.51, 'train_loss': 0.9161754014756944, 'epoch': 4.95}


100%|██████████| 23/23 [00:00<00:00, 36.86it/s]

Accuracy: 0.74, F1-Score: 0.68


In [20]:
model.save_pretrained('./fine_tuned_bert_mini_arabic')
tokenizer.save_pretrained('./fine_tuned_bert_mini_arabic')

('./fine_tuned_bert_mini_arabic\\tokenizer_config.json',
 './fine_tuned_bert_mini_arabic\\special_tokens_map.json',
 './fine_tuned_bert_mini_arabic\\vocab.txt',
 './fine_tuned_bert_mini_arabic\\added_tokens.json',
 './fine_tuned_bert_mini_arabic\\tokenizer.json')

In [21]:
df=pd.read_csv(r"C:\Users\noura\OneDrive\Desktop\university\arabic sentiment analysis project\notebooks\elmenus_reviews_with_sentiment.csv")
df.head()

,source,text,rating,reviewer,time_stamp,text_translated,sentiment
0,Elmenus,"the delivery was very very late, the estimated...",2,Judy Asal,8 days ago,كان التسليم متأخراً للغاية ، وكان الوقت المقدر...,negative
1,Elmenus,Lots of sauce and not much beef. The photo sho...,2,Patrick bradley,15 days ago,الكثير من الصلصة وليس الكثير من اللحم البقري. ...,negative
2,Elmenus,No sushi menu!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! ...,1,Oleksandr Kumeiko,a month ago,لا قائمة السوشي !!!!!! لا قائمة السوشي !!!!,negative
3,Elmenus,الاكل بارد جدا واتاخر ساعه,1,Abdallah M. Ahmed,a month ago,الاكل بارد جدا واتاخر ساعه,negative
4,Elmenus,الشانتونج كله لازق في بعضه,3,Amir Rafik,a month ago,الشانتونج كله لازق في بعضه,neutral


In [22]:
def preprocess_arabic(text):
    text = str(text)  # Ensure text is string
    text = normalize_alef_maksura_ar(text)
    text = normalize_teh_marbuta_ar(text)
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

In [23]:
def rating_to_sentiment(rating):
    if pd.isna(rating):
        return 'neutral'  # Default for missing ratings
    rating = int(rating)
    if rating <= 2:
        return 'negative'
    elif rating == 3:
        return 'neutral'
    else:
        return 'positive'
    

In [24]:
required_columns = ['text_translated', 'rating']

# Preprocess text_translated and create sentiment column
df['text_translated'] = df['text_translated'].apply(preprocess_arabic)
df['sentiment'] = df['rating'].apply(rating_to_sentiment)



In [26]:
from transformers import pipeline
classifier = pipeline('text-classification', model='./fine_tuned_bert_mini_arabic', tokenizer='./fine_tuned_bert_mini_arabic')

In [27]:
df['predicted_sentiment'] = df['text_translated'].apply(lambda x: classifier(x)[0]['label'].replace('LABEL_0', 'negative').replace('LABEL_1', 'neutral').replace('LABEL_2', 'positive'))

In [28]:
df.head()

,source,text,rating,reviewer,time_stamp,text_translated,sentiment,predicted_sentiment
0,Elmenus,"the delivery was very very late, the estimated...",2,Judy Asal,8 days ago,كان التسليم متأخرا للغايه وكان الوقت المقدر 9...,negative,negative
1,Elmenus,Lots of sauce and not much beef. The photo sho...,2,Patrick bradley,15 days ago,الكثير من الصلصه وليس الكثير من اللحم البقري أ...,negative,negative
2,Elmenus,No sushi menu!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! ...,1,Oleksandr Kumeiko,a month ago,لا قائمه السوشي لا قائمه السوشي,negative,negative
3,Elmenus,الاكل بارد جدا واتاخر ساعه,1,Abdallah M. Ahmed,a month ago,الاكل بارد جدا واتاخر ساعه,negative,negative
4,Elmenus,الشانتونج كله لازق في بعضه,3,Amir Rafik,a month ago,الشانتونج كله لازق في بعضه,neutral,negative


In [29]:
df['predicted_sentiment'].value_counts()

predicted_sentiment
negative    372
positive     82
Name: count, dtype: int64

In [30]:
df.to_csv('elmenus_reviews_predicted.csv', index=False)
print("Created and saved 'elmenus_reviews_predicted.csv' with predictions.")

Created and saved 'elmenus_reviews_predicted.csv' with predictions.


In [32]:
from sklearn.metrics import accuracy_score, f1_score
accuracy = accuracy_score(df['sentiment'], df['predicted_sentiment'])
f1 = f1_score(df['sentiment'], df['predicted_sentiment'], average='weighted', labels=['negative', 'neutral', 'positive'])
print(f"\nAccuracy: {accuracy:.2f}, F1-Score: {f1:.2f}")


Accuracy: 0.69, F1-Score: 0.62
